<a href="https://colab.research.google.com/github/abdulmukit98/Tensorflow-in-practice-specialization/blob/master/Convolutional_Neural_Networks_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring a larger datasets

In [0]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

In [0]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir='/tmp/cats_and_dogs_filtered/train'
validation_dir = '/tmp/cats_and_dogs_filtered/validation'
train_cats_dir='/tmp/cats_and_dogs_filtered/train/cats'
train_dogs_dir='/tmp/cats_and_dogs_filtered/train/dogs'
validation_cats_dir='/tmp/cats_and_dogs_filtered/validation/cats'
validation_dogs_dir='/tmp/cats_and_dogs_filtered/validation/dogs'

In [0]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)
print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

print('total train cat images: ',len(train_cat_fnames))
print('total train dog images: ',len(train_dog_fnames))

print('total validation cat images: ', len(os.listdir(validation_cats_dir)))
print('total validation dog images: ', len(os.listdir(validation_dogs_dir)))

In [0]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows = 4
ncols = 4

pic_index = 0

Display a batch of 8 cat and 8 dogs picture

In [0]:
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, i)
      for i in train_cat_fnames[pic_index-8:pic_index]]

next_dog_pix = [os.path.join(train_dogs_dir, i)
      for i in train_dog_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix + next_dog_pix):
  sp = plt.subplot(nrows, ncols, i+1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

Tensorflow works

In [0]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
model = tf.keras.models.Sequential([
    tfl.Conv2D(16,(3,3),activation='relu',
              input_shape=(150,150,3)),
    tfl.MaxPooling2D(2,2),
    tfl.Conv2D(32,(3,3),activation='relu'),
    tfl.MaxPooling2D(2,2),
    tfl.Conv2D(64,(3,3),activation='relu'),
    tfl.MaxPooling2D(2,2),
    tfl.Flatten(),
    tfl.Dense(512,activation='relu'),
    tfl.Dense(1,activation='sigmoid')
])

model.summary()

from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255 )
test_datagen = ImageDataGenerator( rescale = 1.0/255 )

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150,150)
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,
    class_mode = 'binary',
    target_size = (150,150)
)

In [0]:
history = model.fit_generator(
    train_generator,
    validation_data= validation_generator,
    steps_per_epoch = 100,
    epochs = 15,
    validation_steps = 50,
    verbose = 1
)

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path,
                       target_size = (150,150))
  
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0] > 0:
    print(fn+" is a dog")
  else:
    print(fn+" is a cat")

###Evaluating accuracy and loss

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_no = range(len(acc))

plt.plot(epochs_no, acc)
plt.plot(epochs_no, val_acc)
plt.title('Train and validation accuracy')

plt.figure() #new fig

plt.plot(epochs_no, loss)
plt.plot(epochs_no, val_loss)
plt.title('Train and validation loss')


# Augmentation

### cats versus dog with augmentation

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip
  

In [0]:
import os 
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


import tensorflow.keras.layers as tfl
model = tf.keras.models.Sequential([
  tfl.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
  tfl.MaxPooling2D(2,2),
  tfl.Conv2D(64, (3,3), activation='relu'),
  tfl.MaxPooling2D(2,2),
  tfl.Conv2D(128,(3,3),activation='relu'),
  tfl.MaxPooling2D(2,2),
  tfl.Conv2D(128,(3,3),activation='relu'),
  tfl.MaxPooling2D(2,2),
  tfl.Dropout(0.5),
  tfl.Flatten(),
  tfl.Dense(512, activation='relu'),
  tfl.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(lr=1e-4),
              metrics = ['acc'])

train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
) 
test_datagen = ImageDataGenerator(
    rescale=1.0/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size=20,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode = 'binary'
)


history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 50,
    verbose = 1
)

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'g', label = 'validation acc')
plt.legend() # make label show
plt.title('Training and validation acc')

plt.figure()

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, val_loss, 'g', label= 'val loss')
plt.legend()
plt.title('Training and validation loss')

# Horse and Human with augmentation

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip
  


In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')

local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

train_horse_dir = '/tmp/horse-or-human/horses' 
train_human_dir = '/tmp/horse-or-human/humans'
validation_horse_dir = '/tmp/validation-horse-or-human/horses'
validation_human_dir = '/tmp/validation-horse-or-human/humans'


import tensorflow as tf
import tensorflow.keras.layers as tfl
model = tf.keras.models.Sequential([
        tfl.Conv2D(16,(3,3),activation='relu',
                  input_shape=(300,300,3)),
        tfl.MaxPooling2D(2,2),
        tfl.Conv2D(32,(3,3),activation='relu'),
        tfl.MaxPooling2D(2,2),
        tfl.Conv2D(64,(3,3),activation='relu'),
        tfl.MaxPooling2D(2,2),
        tfl.Conv2D(64,(3,3),activation='relu'),
        tfl.MaxPooling2D(2,2),
        tfl.Conv2D(64,(3,3),activation='relu'),
        tfl.MaxPooling2D(2,2),
        tfl.Flatten(),
        tfl.Dense(512,activation='relu'),
        tfl.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(lr = 1e-4),
              metrics=['acc']
)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
validation_datagen = ImageDataGenerator(
    rescale = 1/255
)

train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human/',
    target_size = (300,300),
    batch_size = 128,
    class_mode = 'binary'
)
validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human/',
      target_size = (300,300),
      batch_size = 32,
      class_mode = 'binary'
)


In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 20,
    verbose = 1,
    validation_data = validation_generator,
    validation_steps = 8
)

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='acc')
plt.plot(epochs, val_acc, 'b', label='val_acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='loss')
plt.plot(epochs, val_loss, 'b', label='val_loss')
plt.legend()

#Transfer Learning
Inception datasets

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  

In [0]:
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(
                        input_shape = (150,150,3),
                        include_top = False,
                        weights = None
)

pre_trained_model.load_weights(local_weights_file)

for i in pre_trained_model.layers:
  i.trainable = False

pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed10')
print('last layer output shape', last_layer.output_shape)

#load all pre trained into (last_output)
last_output = last_layer.output

x = tfl.Flatten()(last_output)
x = tfl.Dense(1024,activation='relu')(x)
x = tfl.Dropout(0.2) (x)
x = tfl.Dense(1,activation='sigmoid') (x)

#models.Model   not models.Sequential
model = tf.keras.models.Model(pre_trained_model.input, x)

from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr = 1e-4),
              loss = 'binary_crossentropy',
              metrics = ['acc']
              )


Applying model to cats vs dogs datasets

In [0]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150,150)
)

validation_datagen = ImageDataGenerator(rescale = 1/255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150,150)
)

In [0]:
#Now we use our transfer learning model
history = model.fit_generator(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 20,
    validation_steps = 50,
    verbose = 1
)

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.title('train and validation acc')
plt.plot(epochs, acc, 'r', label = 'Train_acc')
plt.plot(epochs, val_acc, 'b', label = 'Val_acc')
plt.legend()



#Multiclass Classifications
Rock paper scissors

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

In [0]:
import os
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_dir = "/tmp/rps"
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    class_mode = 'categorical'
)

validation_dir = '/tmp/rps-test-set'
validation_datagen = ImageDataGenerator(
    rescale = 1/255
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    class_mode = 'categorical'
)

import tensorflow.keras.layers as tfl
model = tf.keras.models.Sequential([
    tfl.Conv2D(64, (3,3), activation = 'relu',
               input_shape = (150, 150, 3)),
    tfl.MaxPooling2D(2,2),
    tfl.Conv2D(64, (3,3), activation='relu'),
    tfl.MaxPooling2D(2,2),
    tfl.Conv2D(128, (3,3), activation= 'relu'),
    tfl.MaxPooling2D(2,2),
    tfl.Conv2D(128, (3,3), activation= 'relu'),
    tfl.MaxPooling2D(2,2),
    tfl.Flatten(),
    tfl.Dense(512, activation= 'relu'),
    tfl.Dropout(0.5),
    tfl.Dense(3, activation='softmax')
])
model.summary()  
model.compile (
    loss = 'categorical_crossentropy',
    optimizer = 'rmsprop',  #no call function, direct 
    metrics = ['acc']
)


In [0]:
history = model.fit_generator(
    train_generator,
    epochs = 25,
    validation_data = validation_generator,
    verbose = 1
)

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='acc')
plt.plot(epochs, val_acc, 'b', label ='val_acc')
plt.legend()

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload() 

for i in uploaded.keys():
  path = i
  img = image.load_img(path,
                       target_size = (150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  print(i)
  print(classes)